```
Licensed to the Apache Software Foundation (ASF) under one
or more contributor license agreements.  See the NOTICE file
distributed with this work for additional information
regarding copyright ownership.  The ASF licenses this file
to you under the Apache License, Version 2.0 (the
"License"); you may not use this file except in compliance
with the License.  You may obtain a copy of the License at
  http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an
"AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
KIND, either express or implied.  See the License for the
specific language governing permissions and limitations
under the License.
```

In [1]:
import os

import geopandas as gpd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when, explode, hex



from sedona.spark import *
from utilities import getConfig

## Setup Sedona environment

In [ ]:
config = SedonaContext.builder() .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.0_2.12:1.5.0,'
           'org.datasyslab:geotools-wrapper:1.5.0-28.2'). \
    getOrCreate()

sedona = SedonaContext.create(config)
sc = sedona.sparkContext
sc.setSystemProperty("sedona.global.charset", "utf8")

## Read countries shapefile into a Sedona DataFrame 
Data link: https://www.naturalearthdata.com/downloads/50m-cultural-vectors/

In [ ]:
countries = ShapefileReader.readToGeometryRDD(sc, "data/ne_50m_admin_0_countries_lakes/")
countries_df = Adapter.toDf(countries, sedona)
countries_df.createOrReplaceTempView("country")
countries_df.printSchema()

## Read airports shapefile into a Sedona DataFrame 
Data link: https://www.naturalearthdata.com/downloads/50m-cultural-vectors/

In [4]:
airports = ShapefileReader.readToGeometryRDD(sc, "data/ne_50m_airports/")
airports_df = Adapter.toDf(airports, sedona)
airports_df.createOrReplaceTempView("airport")
airports_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- scalerank: string (nullable = true)
 |-- featurecla: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- abbrev: string (nullable = true)
 |-- location: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- wikipedia: string (nullable = true)
 |-- natlscale: string (nullable = true)





## Run Spatial Join using SQL API

In [5]:
result = sedona.sql("SELECT c.geometry as country_geom, c.NAME_EN, a.geometry as airport_geom, a.name FROM country c, airport a WHERE ST_Contains(c.geometry, a.geometry)")

## Run Spatial Join using RDD API

In [6]:
airports_rdd = Adapter.toSpatialRdd(airports_df, "geometry")
# Drop the duplicate name column in countries_df
countries_df = countries_df.drop("NAME")
countries_rdd = Adapter.toSpatialRdd(countries_df, "geometry")

airports_rdd.analyze()
countries_rdd.analyze()

# 4 is the num partitions used in spatial partitioning. This is an optional parameter
airports_rdd.spatialPartitioning(GridType.KDBTREE, 4)
countries_rdd.spatialPartitioning(airports_rdd.getPartitioner())

buildOnSpatialPartitionedRDD = True
usingIndex = True
considerBoundaryIntersection = True
airports_rdd.buildIndex(IndexType.QUADTREE, buildOnSpatialPartitionedRDD)

result_pair_rdd = JoinQueryRaw.SpatialJoinQueryFlat(airports_rdd, countries_rdd, usingIndex, considerBoundaryIntersection)

result2 = Adapter.toDf(result_pair_rdd, countries_rdd.fieldNames, airports.fieldNames, sedona)

result2.createOrReplaceTempView("join_result_with_all_cols")
# Select the columns needed in the join
result2 = sedona.sql("SELECT leftgeometry as country_geom, NAME_EN, rightgeometry as airport_geom, name FROM join_result_with_all_cols")

[('3.0', '2.12', '1.5.0')]


## Print spatial join results

In [7]:
# The result of SQL API
result.show()
# The result of RDD API
result2.show()

23/10/30 20:48:30 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


+--------------------+--------------------+--------------------+--------------------+
|        country_geom|             NAME_EN|        airport_geom|                name|
+--------------------+--------------------+--------------------+--------------------+
|MULTIPOLYGON (((1...|Taiwan           ...|POINT (121.231370...|Taoyuan          ...|
|MULTIPOLYGON (((5...|Netherlands      ...|POINT (4.76437693...|Schiphol         ...|
|POLYGON ((103.969...|Singapore        ...|POINT (103.986413...|Singapore Changi ...|
|MULTIPOLYGON (((-...|United Kingdom   ...|POINT (-0.4531566...|London Heathrow  ...|
|MULTIPOLYGON (((-...|United States of ...|POINT (-149.98172...|Anchorage Int'l  ...|
|MULTIPOLYGON (((-...|United States of ...|POINT (-84.425397...|Hartsfield-Jackso...|
|MULTIPOLYGON (((1...|People's Republic...|POINT (116.588174...|Beijing Capital  ...|
|MULTIPOLYGON (((-...|Colombia         ...|POINT (-74.143371...|Eldorado Int'l   ...|
|MULTIPOLYGON (((6...|India            ...|POINT (72.8

## Group airports by country

In [8]:
# result.createOrReplaceTempView("result")
result2.createOrReplaceTempView("result")
groupedresult = sedona.sql("SELECT c.NAME_EN, c.country_geom, count(*) as AirportCount FROM result c GROUP BY c.NAME_EN, c.country_geom")
groupedresult.show()
groupedresult.createOrReplaceTempView("grouped_result")

+--------------------+--------------------+------------+
|             NAME_EN|        country_geom|AirportCount|
+--------------------+--------------------+------------+
|Cuba             ...|MULTIPOLYGON (((-...|           1|
|Mexico           ...|MULTIPOLYGON (((-...|          12|
|Panama           ...|MULTIPOLYGON (((-...|           1|
|Nicaragua        ...|POLYGON ((-83.157...|           1|
|Honduras         ...|MULTIPOLYGON (((-...|           1|
|Colombia         ...|MULTIPOLYGON (((-...|           4|
|United States of ...|MULTIPOLYGON (((-...|          35|
|Ecuador          ...|MULTIPOLYGON (((-...|           1|
|The Bahamas      ...|MULTIPOLYGON (((-...|           1|
|Peru             ...|POLYGON ((-69.965...|           1|
|Guatemala        ...|POLYGON ((-92.235...|           1|
|Canada           ...|MULTIPOLYGON (((-...|          15|
|Venezuela        ...|MULTIPOLYGON (((-...|           3|
|Argentina        ...|MULTIPOLYGON (((-...|           3|
|Bolivia          ...|MULTIPOLY

## Visualize the number of airports in each country

In [9]:
sedona_kepler_map = SedonaKepler.create_map(df=groupedresult, name="AirportCount", config=getConfig())
sedona_kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'ikzru0t', 'type': …

## Visualize Uber H3 cells using SedonaKepler
The following tutorial depicts how Uber H3 cells can be generated using Sedona and visualized using SedonaKepler.

### Generate H3 cell IDs
[ST_H3CellIDs](https://sedona.apache.org/1.5.0/api/flink/Function/#st_h3cellids) can be used to generated cell IDs for given geometries

In [10]:
h3_df = sedona.sql("SELECT g.NAME_EN, g.country_geom, ST_H3CellIDs(g.country_geom, 3, false) as h3_cellID from grouped_result g")
h3_df.show(2)

+--------------------+--------------------+--------------------+
|             NAME_EN|        country_geom|           h3_cellID|
+--------------------+--------------------+--------------------+
|Cuba             ...|MULTIPOLYGON (((-...|[5911955825051566...|
|Mexico           ...|MULTIPOLYGON (((-...|[5918915733655388...|
+--------------------+--------------------+--------------------+
only showing top 2 rows



### Since each geometry can have multiple H3 cell IDs, let's explode the generated H3 cell ID array to get individual cells

In [11]:
exploded_h3 = h3_df.select(h3_df.NAME_EN, h3_df.country_geom, explode(h3_df.h3_cellID).alias("h3"))
exploded_h3.show(2)

+--------------------+--------------------+------------------+
|             NAME_EN|        country_geom|                h3|
+--------------------+--------------------+------------------+
|Cuba             ...|MULTIPOLYGON (((-...|591195582505156607|
|Cuba             ...|MULTIPOLYGON (((-...|591195513785679871|
+--------------------+--------------------+------------------+
only showing top 2 rows



### Convert generated long H3 cell ID to a hex cell ID
SedonaKepler accepts each H3 cell ID as a hexadecimal to automatically visualize them. Also, let us sample the data to be able to visualize sparse cells on the map.

In [12]:
exploded_h3 = exploded_h3.sample(0.3)
exploded_h3.createOrReplaceTempView("exploded_h3")
hex_exploded_h3 = exploded_h3.select(exploded_h3.NAME_EN, hex(exploded_h3.h3).alias("ex_h3"))
hex_exploded_h3.show(2)

+--------------------+---------------+
|             NAME_EN|          ex_h3|
+--------------------+---------------+
|Cuba             ...|83458EFFFFFFFFF|
|Cuba             ...|834590FFFFFFFFF|
+--------------------+---------------+
only showing top 2 rows



### Visualize using SedonaKepler
Now, simply provide the final df to SedonaKepler.create_map and you can automagically visualize the H3 cells on the map!

In [13]:
sedona_kepler_h3 = SedonaKepler.create_map(df=hex_exploded_h3, name="h3")
sedona_kepler_h3

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'h3': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, …